In [4]:
from newspaper import fulltext, Article
from datetime import date, timedelta
import requests
import pandas as pd
from dateutil import parser
import pandas as pd
import numpy as np
import time
import requests
from bs4 import BeautifulSoup
import re
from selectorlib import Extractor
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from multiprocessing.dummy import Pool as ThreadPool
from datetime import datetime

# Web Scraper 

Scrapes all of the links for a company from 2015 - Present.

In [5]:
#Keep in main notebook for easier error handling. 

def fetch_articles(company):    
    now = datetime.now()
    dt_string = now.strftime("%d-%m-%Y_%I-%M-%S%p")
    
    dates = [('#tp_2', dt_string)]

    for date_button, date in dates:

        ChromeOptions = webdriver.ChromeOptions()
        ChromeOptions.add_argument('--incognito')
        ChromeOptions.add_argument('--headless')
        driver = webdriver.Chrome(ChromeDriverManager().install(), options=ChromeOptions)
        time.sleep(2)
        print('navigating to newslookup..')
        news_look = "https://www.newslookup.com"
        time.sleep(1)
        driver.get(news_look)
        time.sleep(3)

        driver.find_element_by_css_selector('#lookup').send_keys(company)

        #search button
        driver.find_element_by_css_selector('#form-group > div > span > button').click()

        #time period
        driver.find_element_by_css_selector('#timeperiod').click()

        #2020--------------------------------------------------
        driver.find_element_by_css_selector(date_button).click()

        #scroll down
        for loaded_page in range(10000):
            try:
                print(f'{loaded_page}')
                time.sleep(.90)
                driver.find_element_by_tag_name('body').send_keys(Keys.END)
                time.sleep(.90)
                driver.find_element_by_css_selector('#more-btn').click()
            except ElementNotInteractableException: 
                print('no more pages')
                break
            except (TimeoutException, ElementClickInterceptedException) as e: #extraneous errors usually fixed after retry.
                continue
            except NoSuchElementException:
                break

        results_list = [] #list of all the links and associated publish dates. 

        raw_html = driver.page_source
        extracted_text = Extractor.from_yaml_string("""
        card:
            css: 'div#results'
            xpath: null
            type: Text
            children:
                title:
                    css: 'a.title:nth-of-type(n+4)'
                    xpath: null
                    multiple: true
                    type: Link
                date:
                    css: span.stime
                    xpath: null
                    multiple: true
                    type: Text

         """)
        raw_data = extracted_text.extract(raw_html)

        #combines the links with their appropriate publish dates (dates in UTC format at this point.)
        results = list(zip(raw_data['card']['title'],raw_data['card']['date'][1:]))
        for entry in results:
            results_list.append(entry)

        def getTxt(lst):
            url, posttime = lst
            article = Article(url)
            article.download()
            try:
                article.parse()
                text = article.text
                auth = article.authors
                source = lst
                summ = article.summary
                titl = article.title
                dic = {'time':posttime,'source':url, 'author':auth, 'fulltext':text, 'summary':summ, 'title':titl}

                return dic
            except Exception as e:
                return None #we can remove null values from results list.
            
        print("Downloading Articles...")    
        pool = ThreadPool(15)
        # open the urls in their own threads
        # and return the results
        raw_output = pool.map(getTxt, results_list)
        results = [record for record in raw_output if record] #only return a list of truthy values. 

        # close the pool and wait for the work to finish 
        pool.close() 
        pool.join()   
        
        #compile and save the final dataframe.
        final_df = pd.DataFrame(results)
        final_df = final_df.drop_duplicates(subset='fulltext')
        final_df.to_csv('/Volumes/pimyllifeupshare/{}_{}.csv'.format(company, date))
        driver.close()



In [6]:
fetch_articles('Goldman Sachs')

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/brendanferris/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


 
navigating to newslookup..
0
1
2
no more pages
